In [15]:
import json
import time

import numpy as np
import pandas as pd
import requests
import bs4
import logging

In [3]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

In [13]:
base_url = 'https://market.yandex.ru/catalog--mobilnye-telefony-otzyvy-pokupatelei/54726/list?show-reviews=1&local-offers-first=0&page='

pages = []
for i in range(1, 51):
    logging.info(f'Loading page {i}')
    url = base_url + str(i)
    resp = requests.get(url, headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.85 YaBrowser/21.11.1.932 Yowser/2.5 Safari/537.36',
        'Cookie': '',
    })
    if not resp.ok or '/captcha_smart.min.css' in resp.text:
        logging.warning(f'Error for page {i}: {resp.status_code}')
        time.sleep(5)
        continue
    pages.append(resp.text)
    time.sleep(1)

INFO: Loading page 1
INFO: Loading page 2
INFO: Loading page 3
INFO: Loading page 4
INFO: Loading page 5
INFO: Loading page 6
INFO: Loading page 7
INFO: Loading page 8
INFO: Loading page 9
INFO: Loading page 10
INFO: Loading page 11
INFO: Loading page 12
INFO: Loading page 13
INFO: Loading page 14
INFO: Loading page 15
INFO: Loading page 16
INFO: Loading page 17
INFO: Loading page 18
INFO: Loading page 19
INFO: Loading page 20
INFO: Loading page 21
INFO: Loading page 22
INFO: Loading page 23
INFO: Loading page 24
INFO: Loading page 25
INFO: Loading page 26
INFO: Loading page 27
INFO: Loading page 28
INFO: Loading page 29
INFO: Loading page 30
INFO: Loading page 31
INFO: Loading page 32
INFO: Loading page 33
INFO: Loading page 34
INFO: Loading page 35
INFO: Loading page 36
INFO: Loading page 37
INFO: Loading page 38
INFO: Loading page 39
INFO: Loading page 40
INFO: Loading page 41
INFO: Loading page 42
INFO: Loading page 43
INFO: Loading page 44
INFO: Loading page 45
INFO: Loading page 

In [17]:
with open('market-pages.json', 'w') as fp:
    json.dump(pages, fp)

In [4]:
with open('market-pages.json') as fp:
    pages = json.load(fp)

In [12]:
def get_node_text(result_set):
    if len(result_set):
        item = result_set
        if not getattr(item, 'find_all', None):
            item = result_set[0]
        texsts = item.find_all(string=True)
        return ' '.join(texsts).strip()
    return ''

In [16]:
def get_market_page_reviews(p):
    soup = bs4.BeautifulSoup(p, 'lxml')
    review_blocks = soup.select('[data-zone-name=product-review]')
    
    result = []
    for review_block in review_blocks:
        rating = int(review_block.select('.autotest-RatingStars')[0].attrs['data-rate'])
        text_blocks = review_block.select('dd')
        text_parts = []
        for text_part in text_blocks[1:]:
            text_parts.append(get_node_text(text_part))
        result.append({'rating': rating, 'text': '\n'.join(text_parts)})
    
    return result
        

rating_data = []
for page in pages:
    rating_data.extend(get_market_page_reviews(page))
rating_data = pd.DataFrame(rating_data)

In [18]:
rating_data.rating.value_counts()

1    284
4     38
2      8
5      6
3      2
Name: rating, dtype: int64

In [21]:
rating_data['positive'] = 0
rating_data.loc[rating_data.rating >= 4, 'positive'] = 1

rating_data.positive.value_counts()

0    294
1     44
Name: positive, dtype: int64